In [1]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


### Build a Yiled Curve

In [2]:
### de-serialize a pre-built model
path = 'serialized/yc_model.pickle'
yc_model : Model = qfReadModelFromFile(path)
# rebuild model for special dates
value_date = qfDisplayModelValueDate(yc_model)
data_collection = qfGetDataCollectionFromModel(yc_model)
build_method_collection = qfGetBuildMethodCollection(yc_model)
value_date_last_date = '2026-05-26'
yc_model_last_date = qfCreateModel(value_date_last_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date_expired = '2026-05-27'
yc_model_expired = qfCreateModel(value_date_expired, 'YIELD_CURVE', data_collection, build_method_collection)

### Build RFR Swap and get Fixed and Floating Leg

In [3]:
effective_date = '2026-05-26'
termination_date = '2026-08-26'
term = '3M'
pay_offset = '2D'
on_index = 'SOFR-1B'
fixed_rate = 0.04
pay_or_rec = 'pay'
notional = 1e6
accrual_peroid = '1Y'
accrual_basis = 'ACT/360'
floating_leg_accrual_period = '1Y'
business_day_convention = 'F'
holiday_convention = 'USGS'
spread = 0.005
compounding_method = 'compound'

product_rfr_swap = qfCreateProductRFRSwap(
    effective_date,
    term,
    pay_offset,
    on_index,
    fixed_rate,
    pay_or_rec,
    notional,
    accrual_peroid,
    accrual_basis,
    floating_leg_accrual_period,
    business_day_convention,
    holiday_convention,
    spread,
    compounding_method)

display(qfDisplayProduct(product_rfr_swap))

,Name,Value
0,Product Type,PRODUCT_RFR_SWAP
1,Notional,1000000.0
2,Currency,USD
3,Long Or Short,LONG
4,Effective Date,2026-05-26
5,Termination Date,2026-08-26
6,Payment Offset,2D
7,ON Index,SOFRON Actual/360
8,Fixed Rate,0.04
9,Pay Or Receive,PAY


In [4]:
fixed_leg_stream = product_rfr_swap.fixed_leg_
float_leg_stream = product_rfr_swap.floating_leg_
print("Fixed leg type:", fixed_leg_stream.product_type)
print("Float leg type:", float_leg_stream.product_type)
print("Fixed leg n_cashflows:", fixed_leg_stream.num_cashflows())
print("Float leg n_cashflows:", float_leg_stream.num_cashflows())

Fixed leg type: PRODUCT_PORTFOLIO
Float leg type: PRODUCT_PORTFOLIO
Fixed leg n_cashflows: 1
Float leg n_cashflows: 1


In [5]:
idx = 0
display(qfDisplayProduct(product_rfr_swap.floating_leg_cash_flow(idx)))
display(qfDisplayProduct(product_rfr_swap.fixed_leg_cash_flow(idx)))

,Name,Value
0,Product Type,PRODUCT_OVERNIGHT_INDEX_CASHFLOW
1,Notional,1000000.0
2,Currency,USD
3,Long Or Short,LONG
4,Effective Date,2026-05-26
5,Termination Date,2026-08-26
6,ON Index,SOFRON Actual/360
7,Compounding Method,COMPOUND
8,Spread,0.005
9,Payment Date,2026-08-28


,Name,Value
0,Product Type,PRODUCT_FIXED_ACCRUED
1,Notional,-1000000.0
2,Currency,USD
3,Long Or Short,SHORT
4,Effective Date,2026-05-26
5,Termination Date,2026-08-26
6,Accrual Basis,ACT/360
7,Payment Date,2026-08-28
8,Business Day Convention,F
9,Holiday Convention,USGS


### Build Valuation Parameter Collection

In [6]:
### create funding index valuation parameters
vp_type = 'FUNDING INDEX PARAMETER'
content = {'Funding Index' : 'SOFR-1B'}
fi_vp = qfCreateValuationParameters(vp_type, content)
vp_collection = qfCreateValuationParametersCollection([fi_vp])

### Test Interest Rate Stream Valuation

In [7]:
### test pv and cash for fixed leg
fixed_pv = qfCreateValueReport(yc_model, fixed_leg_stream, vp_collection, 'pvdetailed')
display(fixed_pv.display())

fixed_pv_last = qfCreateValueReport(yc_model_last_date, fixed_leg_stream, vp_collection, 'pvdetailed')
display(fixed_pv_last.display())

# fixed_pv_exp = qfCreateValueReport(yc_model_expired, fixed_leg_stream, vp_collection, 'pvdetailed')
# display(fixed_pv_exp.display())

,Currency,Type,Value
0,USD,PV,-10003.680155
1,USD,CASH,0.000000


,Currency,Type,Value
0,USD,PV,-10143.438197
1,USD,CASH,0.000000


In [8]:
### test pv and cash for floating leg
float_pv = qfCreateValueReport(yc_model, float_leg_stream, vp_collection, 'pvdetailed')
display(float_pv.display())

float_pv_last = qfCreateValueReport(yc_model_last_date, float_leg_stream, vp_collection, 'pvdetailed')
display(float_pv_last.display())

# float_pv_exp = qfCreateValueReport(yc_model_expired, float_leg_stream, vp_collection, 'pvdetailed')
# display(float_pv_exp.display())

,Currency,Type,Value
0,USD,PV,9521.720823
1,USD,CASH,0.000000


,Currency,Type,Value
0,USD,PV,8799.735545
1,USD,CASH,0.000000


In [9]:
### test risk for fixed leg
fixed_risk = qfCreateValueReport(yc_model, fixed_leg_stream, vp_collection, 'firstorderrisk')
display(fixed_risk)
fixed_risk_last = qfCreateValueReport(yc_model_last_date, fixed_leg_stream, vp_collection, 'firstorderrisk')
display(fixed_risk_last)

In [10]:
### test risk for floating leg
float_risk = qfCreateValueReport(yc_model, float_leg_stream, vp_collection, 'firstorderrisk')
display(float_risk)
float_risk_last = qfCreateValueReport(yc_model_last_date, float_leg_stream, vp_collection, 'firstorderrisk')
display(float_risk_last)

In [11]:
### test cf report for fixed leg
fixed_cf = qfCreateValueReport(yc_model, fixed_leg_stream, vp_collection, 'cashflowsreport')
display(fixed_cf.display())
fixed_cf_last = qfCreateValueReport(yc_model_last_date, fixed_leg_stream, vp_collection, 'cashflowsreport')
display(fixed_cf_last.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_PORTFOLIO,ValuationEngineInterestRateStream,0,0,-1.0,1000000.0,"August 28th, 2026",-10222.222222,-10003.680155,0.978621,"May 26th, 2026","August 26th, 2026",0.255556,FIXED,0.04


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_PORTFOLIO,ValuationEngineInterestRateStream,0,0,-1.0,1000000.0,"August 28th, 2026",-10222.222222,-10143.438197,0.992293,"May 26th, 2026","August 26th, 2026",0.255556,FIXED,0.04


In [12]:
### test cf report for floating leg
float_cf = qfCreateValueReport(yc_model, float_leg_stream, vp_collection, 'cashflowsreport')
display(float_cf.display())
float_cf_last = qfCreateValueReport(yc_model_last_date, float_leg_stream, vp_collection, 'cashflowsreport')
display(float_cf_last.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,FIXING_DATE,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_PORTFOLIO,ValuationEngineInterestRateStream,0,0,1.0,1000000.0,"August 28th, 2026",9729.733926,9521.720823,0.978621,"August 26th, 2026","May 26th, 2026","August 26th, 2026",0.255556,SOFRON Actual/360,0.033073


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,FIXING_DATE,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_PORTFOLIO,ValuationEngineInterestRateStream,0,0,1.0,1000000.0,"August 28th, 2026",8868.083039,8799.735545,0.992293,"August 26th, 2026","May 26th, 2026","August 26th, 2026",0.255556,SOFRON Actual/360,0.029701


### Test RFR Swap

In [16]:
swap_rate = qfCreateValueReport(yc_model, product_rfr_swap, vp_collection, 'parrateorspread')
print(f'The par swap rate is {swap_rate}.')

swap_pv = qfCreateValueReport(yc_model, product_rfr_swap, vp_collection, 'pvdetailed')
display(swap_pv.display())

swap_pv_last = qfCreateValueReport(yc_model_last_date, product_rfr_swap, vp_collection, 'pvdetailed')
display(swap_pv_last.display())

The par swap rate is 0.038072871883377844.


,Currency,Type,Value
0,USD,PV,-481.959332
1,USD,CASH,0.000000


,Currency,Type,Value
0,USD,PV,-1343.702653
1,USD,CASH,0.000000


In [14]:
swap_risk = qfCreateValueReport(yc_model, product_rfr_swap, vp_collection, 'firstorderrisk')
display(swap_risk)
swap_risk_last = qfCreateValueReport(yc_model_last_date, product_rfr_swap, vp_collection, 'firstorderrisk')
display(swap_risk_last)

In [15]:
swap_cf = qfCreateValueReport(yc_model, product_rfr_swap, vp_collection, 'cashflowsreport')
display(swap_cf.display())

swap_cf_last = qfCreateValueReport(yc_model_last_date, product_rfr_swap, vp_collection, 'cashflowsreport')
display(swap_cf_last.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_RFR_SWAP,ValuationEngineProductRfrSwap,0,0,-1.0,1000000.0,"August 28th, 2026",-10222.222222,-10003.680155,0.978621,"May 26th, 2026","August 26th, 2026",0.255556,FIXED,0.040000
1,PRODUCT_RFR_SWAP,ValuationEngineProductRfrSwap,1,0,1.0,1000000.0,"August 28th, 2026",9729.733926,9521.720823,0.978621,"May 26th, 2026","August 26th, 2026",0.255556,SOFRON Actual/360,0.033073


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,START_DATE,END_DATE,ACCRUED,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_RFR_SWAP,ValuationEngineProductRfrSwap,0,0,-1.0,1000000.0,"August 28th, 2026",-10222.222222,-10143.438197,0.992293,"May 26th, 2026","August 26th, 2026",0.255556,FIXED,0.040000
1,PRODUCT_RFR_SWAP,ValuationEngineProductRfrSwap,1,0,1.0,1000000.0,"August 28th, 2026",8868.083039,8799.735545,0.992293,"May 26th, 2026","August 26th, 2026",0.255556,SOFRON Actual/360,0.029701
